In [2]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

### Test Data
We will be using the Ordnance Survey Open Roads dataset converted into a graph.  
A sample is shown below:

![Road_Graph](./img/road_graph.png)

This is a weighted graph, using the edge distance in metres.  
__Note__: The Open Roads dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  
*Contains OS data © Crown copyright and database right (2019)*  
Licensed under [Open Government Licence (OGL) V3](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/)

### Read the source data and adjust the vertex IDs

In [3]:
# Test file  - Read OS Open Roads as a graph. (Store dataset in ./data folder)
datafile='./data/gb_road_graph_20190523.csv'

In [8]:
# Read the data file with lenght of edge in metres as weight
cols = ["src", "dst","length"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32"),
        ("length", "float64"),
        ])

gdf = cudf.read_csv(datafile, names=cols, dtype=list(dtypes.values()) ,skiprows=1)

In [9]:
gdf

<cudf.DataFrame ncols=3 nrows=7347806 >

In [10]:
# Display the graph adjacency matrix
print(gdf)

    src  dst  length
0   16   20   162.0
1   20   16   162.0
2   56   55   855.0
3   55   56   855.0
4   72   22   382.0
5   22   72   382.0
6   72   71   270.0
7   71   72   270.0
8   17   18    48.0
9   18   17    48.0
[7347796 more rows]


In [5]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [7]:
gdf

<cudf.DataFrame ncols=5 nrows=7347806 >

In [6]:
# Display the graph adjacency matrix
print(gdf)

    src  dst  length  src_0  dst_0
0   16   20   162.0     15     19
1   20   16   162.0     19     15
2   56   55   855.0     55     54
3   55   56   855.0     54     55
4   72   22   382.0     71     21
5   22   72   382.0     21     71
6   72   71   270.0     71     70
7   71   72   270.0     70     71
8   17   18    48.0     16     17
9   18   17    48.0     17     16
[7347796 more rows]


### Create the Road Graph and call SSSP

In [31]:
%%time
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"],gdf['length'])

CPU times: user 3.22 ms, sys: 1.9 ms, total: 5.12 ms
Wall time: 5.11 ms


In [51]:
%%time
# Call cugraph.sssp to get the distances from vertex 2225573 (Lusty Glaze Road, Newquay):
df = cugraph.sssp(G,2225573)

CPU times: user 1.81 s, sys: 893 ms, total: 2.71 s
Wall time: 2.7 s


### Calculate 500 miles walking distance

In [59]:
# Calculate 500 miles distance (804672 metres)
print(df.query("distance==804672.00")[1:20])

    vertex  distance
233618  233618  804672.0
266641  266641  804672.0
278509  278509  804672.0
302535  302535  804672.0
304584  304584  804672.0
305391  305391  804672.0
305553  305553  804672.0
305556  305556  804672.0
334746  334746  804672.0
371325  371325  804672.0


___
Copyright (c) 2019, NVIDIA CORPORATION.

Adapted by John Murray

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___